### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics


import pandas as pd
import numpy as np
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.utils.data as data
from PIL import Image
import torch.nn.functional as F

In [2]:
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"GPU is available with {num_gpus} device(s).")

    # Get the name of the current GPU
    current_gpu = torch.cuda.get_device_name(0)  # Assuming the first GPU is used
    print(f"Current GPU: {current_gpu}")
else:
    print("GPU is not available. Using CPU.")

GPU is not available. Using CPU.


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the first available GPU
else:
    device = torch.device("cpu")  # If no GPU is available, use CPU

In [ ]:
class SolarPanelDataset(data.Dataset):

    #data_dir: The directory containing the image files
    #image_shape: The shape of the images (height, width, channels)
    #augmentation_copies: The number of copies of each image to create for data augmentation

    def __init__(self, df, data_dir, image_shape=(3, 256, 256), augmentation_copies=3):

        self.copies = 1 + augmentation_copies
        self.image_shape = image_shape
        self.df = df #df: A pandas DataFrame containing the image filenames and labels
        self.labels = []
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
        ])
        self.augmentation_transform = self.make_augmentation_transform()
        self.dataset_size = len(df)
        self.images = torch.zeros((self.dataset_size, *self.image_shape), dtype=torch.float32)
        for i, img_name in enumerate(self.df['original_title']):
            img_path = os.path.join(data_dir, img_name)
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            self.images[i] = image
            label = df.iloc[i]['loss_percentage']
            self.labels.append(label)  # Add corresponding label here

    def __len__(self):
        return len(self.images)*self.copies

    def make_augmentation_transform(self):
        return transforms.Compose([
            transforms.RandomResizedCrop((self.image_shape[1], self.image_shape[2])),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.RandomVerticalFlip(p=0.2),
            transforms.RandomRotation(45),
            transforms.ColorJitter(brightness=(0.3, 2), contrast=0, saturation=0, hue=0),
            #transforms.RandomGrayscale(p=0.1)
        ])

    def __getitem__(self, idx):
        if (idx >= len(self.images) ):
            image = self.images[idx % len(self.images)]
            label = self.labels[idx % len(self.labels)]
            return self.augmentation_transform(image), torch.tensor(label, dtype=torch.float32)


        image = self.images[idx]
        label = self.labels[idx]
        return image, torch.tensor(label, dtype=torch.float32)